<h1>Join<h1>

In [1]:
import pandas as pd
import sqlite3

<h2>create a connection to the database using the library sqlite3<h2>

In [2]:
conn = sqlite3.connect("../../data/checking-logs.sqlite")

<h2>create a new table datamart in the database by joining the tables pageviews and checker using only one query<h2>

In [3]:
curs = conn.cursor()
curs.execute("DROP TABLE IF EXISTS datamart")
curs.execute('''
    CREATE TABLE datamart AS
        SELECT checker.uid, checker.labname, MIN(timestamp) AS first_commit_ts, min AS first_view_ts
        FROM checker
        LEFT JOIN (
            SELECT uid, MIN(datetime) AS min
            FROM pageviews
            GROUP BY uid 
        ) AS user_first_view
        ON checker.uid = user_first_view.uid
        WHERE status = 'ready' AND numTrials = 1 AND labname IN  ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') AND checker.uid LIKE 'user_%'
        GROUP BY checker.uid, checker.labname
    ''')

<h2>using Pandas methods, create two dataframes: test and control<h2>

In [4]:
datamart = pd.io.sql.read_sql("SELECT * FROM datamart", conn,
                               parse_dates=["first_commit_ts","first_view_ts"])

In [5]:
test = datamart[datamart["first_view_ts"].isna() == False]
control = datamart[datamart["first_view_ts"].isna()]

In [6]:
control["first_view_ts"] = test["first_view_ts"].mean()

/var/folders/v0/x45rbc1s671ft5z86pdqjpvw0000gn/T/ipykernel_73413/175981500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control["first_view_ts"] = test["first_view_ts"].mean()


In [7]:
test.to_sql("test", conn, if_exists="replace", index=False)
control.to_sql("control", conn, if_exists="replace", index=False)


81

<h2>close the connection<h2>

In [8]:
conn.close()